In [1]:
import numpy as np

In [ ]:
with open( 'data/collect_2025.Nov.24_15.47.57.5524625.UTC/_palindrome_state__2025.Nov.24_15.47.57.5549775.UTC.log', 'r' ) as f:
    
    # SE3 matrix constants
    SE3_ROWS = 4
    SE3_COLS = 4
    SE3_SIZE = SE3_ROWS * SE3_COLS

    # get the descriptor line
    keys = f.readline().strip().split( "   " )

    # removes unnecessary '' characters
    keys = [ i for i in keys if len( i ) > 1 ]

    # readline above incremented the iterator - this will grab the data only
    data: dict = {}
    for line in f:
        data_line = line.strip().split( " " )
        data[ int( data_line[1] ) ] = {}

        data[ int( data_line[1] ) ][ keys[0] ] = data_line[0]
        data[ int( data_line[1] ) ][ keys[1] ] = data_line[1]

        for i, key in enumerate( keys[2:] ):
            cols = []
            for col in range( SE3_COLS ):
                cols.append( data_line[ 2 + i * SE3_SIZE + col * SE3_ROWS : 2 + i * SE3_SIZE + ( col + 1 ) * SE3_ROWS ] )
            data[ int( data_line[1] ) ][ key ] = np.array( cols, dtype = np.float64 ).T
        
        if( 'Sensor Pose' in keys and 'Tanker Pose' in keys ):
            print( data[ int( data_line[1] ) ]['Tanker Pose'][:3, 3:], data[ int( data_line[1] ) ]['Sensor Pose'][:3, 3:] )
            so3 = data[ int( data_line[1] ) ]['Sensor Pose'][:3, :3].T @ data[ int( data_line[1] ) ]['Tanker Pose'][:3, :3]
            t_t_s = data[ int( data_line[1] ) ]['Sensor Pose'][:3, :3].T @ ( data[ int( data_line[1] ) ]['Tanker Pose'][:3, 3:] - data[ int( data_line[1] ) ]['Sensor Pose'][:3, 3:] )
            data[ int( data_line[1] ) ]['tanker_in_sensor_frame'] = np.concatenate( [so3, t_t_s], axis = 1 )

[[50.]
 [ 0.]
 [20.]] [[-38.25270844]
 [ 23.0537281 ]
 [283.59738159]]
